# Outlier Analysis
The Density Field (DF) plots from all the catalogues of interest do not give any strong conclusive proof about the viability of one model over another. As a final shot before the Density Field analysis is abandoned, I check if any outlier trend can be found in the data. Finding a conclusive separation in the outlier halo DF analysis is interesting because that will imply that the extreme (outlier) halos from each catalogue do have a significant difference amongst themselves, and if found observationally, it might weakly explain the DLA problem.<br>
The idea is as follows-<br><br>
Of all the Halos of interest, I will find the ones which have a more than average DF in their surrounding. This analysis will happen in 2 ways.
<ol>
    <li>Choose the outliers on the basis of central peak DF and select the halos having the largest value for this.</li>
    <li>Define an average measure of DF over all the points available upto 10*R200b, and choose the ones having the largest value.</li>
</ol>
<br>
This analysis will be done for - 
<ol>
    <li>The one Halo which will have the largest measure in each case described above, from all the catalogues (2 plots for the above 2 methods)</li>
    <li>The top 5 percentile of Halos from each dataset having the largest measure, for both the 2 methods above (2 more plots from here)</li>
    <br>
</ol>
Another interesting problem to solve is to find at what percentile cut do we get a significant separation between the 3 methods. This problem will also be explored in this notebook
